In [ ]:
import pandas as pd
import os

# --- CONFIGURACIÓN ---
# Usamos latin-1 porque ya sabemos que tus archivos originales lo requieren
encoding_origen = 'latin-1'
encoding_destino = 'utf-8-sig' # Ideal para que Power BI reconozca tildes automáticamente

def buscar_archivo(palabra_clave):
    """Busca el archivo CSV correcto en la carpeta"""
    archivos = os.listdir()
    for archivo in archivos:
        if palabra_clave in archivo and archivo.endswith(".csv") and "clean" not in archivo:
            return archivo
    return None

def preparar_datos_para_powerbi():
    print("INICIANDO PREPARACIÓN DE DATOS PARA POWER BI...")
    
    # 1. Carga
    archivos = {
        'clientes': buscar_archivo('clientes'),
        'productos': buscar_archivo('productos'),
        'ventas': buscar_archivo('ventas'),
        'detalle': buscar_archivo('detalle')
    }

    if not all(archivos.values()):
        print(f"Falta algún archivo. Revisa la carpeta. Encontrados: {archivos}")
        return

    df_cli = pd.read_csv(archivos['clientes'], encoding=encoding_origen)
    df_prod = pd.read_csv(archivos['productos'], encoding=encoding_origen)
    df_ven = pd.read_csv(archivos['ventas'], encoding=encoding_origen)
    df_det = pd.read_csv(archivos['detalle'], encoding=encoding_origen)

    # 2. Limpieza y Formato
    print("Limpiando fechas y formatos...")
    
    # Estandarizar Fechas (Formato ISO YYYY-MM-DD es el mejor para PBI)
    df_ven['fecha'] = pd.to_datetime(df_ven['fecha']).dt.strftime('%Y-%m-%d')
    
    # Asegurar que los IDs sean números enteros (sin decimales)
    df_ven['id_cliente'] = df_ven['id_cliente'].astype(int)
    df_det['id_venta'] = df_det['id_venta'].astype(int)
    df_det['id_producto'] = df_det['id_producto'].astype(int)

    # 3. Exportación para Power BI
    # Exportamos tablas separadas para hacer un Modelo Estrella en PBI
    print("Exportando archivos 'Clean'...")
    
    df_cli.to_csv('PBI_Clientes.csv', index=False, encoding=encoding_destino)
    df_prod.to_csv('PBI_Productos.csv', index=False, encoding=encoding_destino)
    df_ven.to_csv('PBI_Ventas.csv', index=False, encoding=encoding_destino)
    df_det.to_csv('PBI_Detalle.csv', index=False, encoding=encoding_destino)
    
    print("¡LISTO! Se han generado 4 archivos 'PBI_...' listos para importar.")

    if __name__ == "__main__":
    preparar_datos_para_powerbi()
